In [1]:
from pyspark.sql.types import StringType
import json
import pandas as pd

def recommend(row):
    d = json.loads(row)
    result = {'uid':d['uid'] , 'pred': '' }
    return str(json.dumps(result))

In [2]:
df = spark.readStream.format("kafka") \
  .option("kafka.bootstrap.servers", "155.138.192.245:9092") \
  .option("subscribe", "quickstart-events") \
  .option("startingOffsets", "latest").load()

In [3]:
df.printSchema()

root
-- key: binary (nullable = true)
-- value: binary (nullable = true)
-- topic: string (nullable = true)
-- partition: integer (nullable = true)
-- offset: long (nullable = true)
-- timestamp: timestamp (nullable = true)
-- timestampType: integer (nullable = true)

In [4]:
df = df.selectExpr("CAST(value AS STRING)")
recommend_udf = udf(recommend, StringType())
df = df.select(recommend_udf("value").alias("value"))

In [5]:
query = df.writeStream.format("kafka")\
  .option("kafka.bootstrap.servers", "155.138.192.245:9092")\
  .option("topic", "recommendation-events")\
  .option("checkpointLocation", "/temp").start().awaitTermination()